In [1]:
import pandas as pd
import numpy as np
import os
import pandas_datareader as pdr
from datetime import datetime


In [2]:
# set time frame
timeframe = '1D'

# start and end date
start_date = datetime(2004,6,1)
end_date = datetime(2009,6,1)

# set tickers
tickers = ['DIS']

# pulling data from pdr 
tickers_df = pdr.DataReader(tickers,'yahoo',start_date,end_date)

In [3]:
# closing prices dataframe
data = pd.DataFrame()

# get closing prices for all tickers in tickers
for ticker in tickers:
    data[ticker]=tickers_df['Adj Close'][ticker]

In [4]:
# add percent change
data['daily_return'] = data['DIS'].pct_change().dropna()

In [13]:
data = data.dropna()
data.head()


,DIS,daily_return,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,vol_trend_long,vol_trend_short,vol_trend_signal,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,bollinger_short,bollinger_signal
Date,,,,,,,,,,,,,,,,,,,
2004-07-01,20.343649,-0.008238,20.104283,20.007995,1.0,0.0,1.0,0.010331,0.010583,1.0,0.0,1.0,19.999220,0.339524,20.338744,19.659696,0.0,-1.0,-1.0
2004-07-02,20.335600,-0.000396,20.124083,20.025153,1.0,0.0,1.0,0.009897,0.010315,1.0,0.0,1.0,20.051528,0.303046,20.354573,19.748482,0.0,0.0,0.0
2004-07-06,19.876900,-0.022557,20.103312,20.017677,1.0,0.0,1.0,0.011765,0.011506,0.0,-1.0,-1.0,20.077279,0.260213,20.337492,19.817066,0.0,0.0,0.0
2004-07-07,19.901045,0.001215,20.086601,20.012003,1.0,0.0,1.0,0.011266,0.011212,0.0,-1.0,-1.0,20.090557,0.242666,20.333223,19.847891,0.0,0.0,0.0
2004-07-08,19.595243,-0.015366,20.046630,19.992414,1.0,0.0,1.0,0.011621,0.011512,0.0,-1.0,-1.0,20.080498,0.259113,20.339611,19.821386,1.0,0.0,1.0


In [6]:
# EWMA Short and Long Windows
short_window = 10
long_window = 50

# Fast and Slow EWMAs from short and long windows
data['ewma_short'] = data['DIS'].ewm(halflife=short_window).mean()
data['ewma_long'] = data['DIS'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
data['crossover_long'] = np.where(data['ewma_short'] > data['ewma_long'], 1.0, 0.0) 
# making sep columns if (where) fast passes close == long 
data['crossover_short'] = np.where(data['ewma_short'] < data['ewma_long'], -1.0, 0.0)
data['crossover_signal'] = data['crossover_long'] + data['crossover_short']


In [7]:
# Volatility - Short and Long Windows
short_vol_window = 10
long_vol_window = 50

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
data['short_vol'] = data['daily_return'].ewm(halflife=short_vol_window).std()
data['long_vol'] = data['daily_return'].ewm(halflife=long_vol_window).std()

# Construct a crossover trading signal
data['vol_trend_long'] = np.where(data['short_vol'] < data['long_vol'], 1.0, 0.0)
data['vol_trend_short'] = np.where(data['short_vol'] > data['long_vol'], -1.0, 0.0) 
data['vol_trend_signal'] = data['vol_trend_long'] + data['vol_trend_short']


In [8]:
data = data.dropna()

In [9]:
# Set bollinger band window
bollinger_window = 20

# Calculate rolling mean and standard deviation
data['bollinger_mid_band'] = data['DIS'].rolling(window=bollinger_window).mean()
data['bollinger_std'] = data['DIS'].rolling(window=20).std() # will use for upper and lower bands of bollinger

# Calculate upper and lowers bands of bollinger band
data['bollinger_upper_band']  = data['bollinger_mid_band'] + (data['bollinger_std'] * 1)
data['bollinger_lower_band']  = data['bollinger_mid_band'] - (data['bollinger_std'] * 1)

# Calculate bollinger band trading signal # signal to calc (doing mean revering strategy)
data['bollinger_long'] = np.where(data['DIS'] < data['bollinger_lower_band'], 1.0, 0.0)
data['bollinger_short'] = np.where(data['DIS'] > data['bollinger_upper_band'], -1.0, 0.0)
data['bollinger_signal'] = data['bollinger_long'] + data['bollinger_short']

data.head()

,DIS,daily_return,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,vol_trend_long,vol_trend_short,vol_trend_signal,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,bollinger_short,bollinger_signal
Date,,,,,,,,,,,,,,,,,,,
2004-06-03,19.289444,-0.018428,19.464237,19.469254,0.0,-1.0,-1.0,0.030233,0.030233,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2004-06-04,19.361872,0.003755,19.427725,19.432963,0.0,-1.0,-1.0,0.021024,0.021311,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2004-06-07,19.635481,0.014131,19.485182,19.484650,1.0,0.0,1.0,0.017858,0.018204,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2004-06-08,19.796427,0.008197,19.556345,19.548746,1.0,0.0,1.0,0.015227,0.015743,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2004-06-09,19.780334,-0.000813,19.600430,19.588694,1.0,0.0,1.0,0.013742,0.014346,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [11]:
data.to_csv('../Data/Disney_Closing_2008.csv',index=True)